In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA, TruncatedSVD
from xgboost import XGBClassifier

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-12 20:30:56.838089. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [4]:
def gini(actual, pred):
    assert (len(actual) == len(pred))
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
    totalLosses = all[:, 0].sum()
    giniSum = all[:, 0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)


def gini_normalized(actual, pred):
    return gini(actual, pred) / gini(actual, actual)

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-12 20:30:57.338636. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [5]:
data = pd.read_csv("../data/train.csv", na_values=[-1, -1.0], index_col="id")
X, y = data.loc[:, data.columns != "target"], data.target

for col in X.columns[X.columns.str.endswith("cat")]:
    X.loc[:, col] = X.loc[:, col].astype("category")
X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

resampler = RandomUnderSampler()
indexes = X_train.index.values.reshape(-1, 1)
index_res, y_res = resampler.fit_sample(indexes, y_train)
X_res = X_train.loc[index_res.flatten(), :]

X_mean = X_res.fillna(X_res.mean())
X_test = X_test.fillna(X_test.mean())

/usr/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-12 20:30:57.359811. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [6]:
float_cols = X_mean.columns[(~X_mean.columns.str.contains("bin|cat")) & (X_mean.dtypes == "float64")]
scaler = RobustScaler()
scaler.fit(X_mean[float_cols])
X_mean[float_cols] = scaler.transform(X_mean[float_cols])
X_test[float_cols] = scaler.transform(X_test[float_cols])

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-12 20:31:04.278576. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


#### Standard 

In [7]:
model = XGBClassifier()
model.fit(X_mean, y_res)
y_pred = model.predict(X_test)
std_gini = gini_normalized(y_test, y_pred)
print(std_gini)

0.193783104922


#### Randomized Search

#### Grid Search 

#### Bayesian Optimization 